In [1]:
import numpy as np
import pretty_midi
import music21 as m21
import os
import copy
import Levenshtein as lev
from io import StringIO
import sys
import pickle
import random

In [2]:
class Measure:
    def __init__(self, measure, instrument):
        self.measure = measure
        self.instrument = instrument
    
    def __repr__(self):
        return f'Measure of {self.instrument}'

In [67]:
path ='../game/data/base'

In [68]:
# read a midi file and load it into a music21 object
def load(file_path):
    return m21.converter.parse(os.path.join(path, file_path))

In [69]:
songs = []
for file in os.listdir(path):
    print(file)
    songs.append(load(file))
    break

4-17 - Summoned Beast Battle (extended).mid


In [70]:
orig_song = songs[0]

In [71]:
midi_tracks = m21.midi.translate.streamHierarchyToMidiTracks(orig_song)
midi_tracks

In [ ]:
instruments = [part.partAbbreviation for part in m21.instrument.partitionByInstrument(orig_song).parts]

In [ ]:
instruments

['Pno']

In [ ]:
instrument_association = {
    'Str': m21.instrument.StringInstrument(),
    'Hp': m21.instrument.Harp(),
    'Hn': m21.instrument.Horn(),
    'Timp': m21.instrument.Timpani(),
    'Ch': m21.instrument.Choir(),
    'Fl': m21.instrument.Flute(),
    'Ob': m21.instrument.Oboe(),
    'Cl': m21.instrument.Clarinet(),
    'Bn': m21.instrument.Bassoon(),
    'Sax': m21.instrument.Saxophone(),
    'Tp': m21.instrument.Trumpet(),
    'Tbn': m21.instrument.Trombone(),
    'Tba': m21.instrument.Tuba(),
    'Perc': m21.instrument.Percussion(),
    'Vln': m21.instrument.Violin(),
    'Vla': m21.instrument.Viola(),
    'Pno': m21.instrument.Piano(),
    # None is for the case where the instrument is not specified
    # and will use any kind of percussion instrument
    None: m21.instrument.Percussion()
}

In [ ]:
measures = {}
for index, inst in enumerate(m21.instrument.partitionByInstrument(orig_song).parts):
    inst.makeMeasures(inPlace=True)
    inst.makeTies(inPlace=True)
    inst.stripTies(inPlace=True, matchByPitch=True)
    if (inst.partAbbreviation == 'Hn'):
        inst.show('text')
    for measure in inst.getElementsByClass(m21.stream.Measure):
        
        if measure.offset not in measures:
            measures[measure.offset] = []
        while len(measures[measure.offset]) <= index:
            measures[measure.offset].append(None)
        measures[measure.offset][index] = Measure(measure, inst.partAbbreviation)

In [ ]:
def add_measure(song_, offset):
    # get the number of measures in the song
    measure_num = len(song_.parts[0].getElementsByClass(m21.stream.Measure))
    measures_in_order = measures[offset]
    for index, measure in enumerate(measures_in_order):
        if measure is None:
            continue
        if index == 0 and measure_num == 0:
            parts[index].insert(0, instrument_association[instrument])
        newMeasure = copy.deepcopy(measure.measure)
        parts[index].insert(measure_num * 4, newMeasure)
    return song_

song = m21.stream.Score()
parts = [m21.stream.Part() for _ in range(len(instrument_association))]
for index, instrument in enumerate(instruments):
    song.insert(0, parts[index])

add_measure(song, 0)
song = m21.midi.translate.prepareStreamForMidi(song)
tup = m21.midi.translate.channelInstrumentData(song)
# get all midi events from the song
song.makeMeasures(inPlace=True)
midi_tracks = m21.midi.translate.streamHierarchyToMidiTracks(song)
midi_events = [midi_track.events for midi_track in midi_tracks]
tup[0]


{0: 1}

In [ ]:
int.from_bytes(midi_events[0][1].data)

500000

In [ ]:
inv_map = {v: k for k, v in tup[0].items()}

In [ ]:
measures_list_ = list(measures.values())

In [ ]:
idx = 0
for index, measure in enumerate(measures_list_):
    idx = index
    if len(measure) != len(instruments):
        break

idx

4

In [ ]:
measures_list = measures_list_[:idx]

In [ ]:
# generate a similarity matrix for the measures
similarity_matrix = np.zeros((len(measures_list), len(measures_list)))
for i in range(len(measures_list)):
    print(f'Calculating similarity for measure {i}\n')
    for j in range(len(measures_list)):
        if i == j:
            similarity_matrix[i][j] = 1
            continue
        if i > j:
            similarity_matrix[i][j] = similarity_matrix[j][i]
            continue
        # calculate the similarity between two measures
        # by calculating the levenstein distance between
        # the two measures in every instrument 
        # and then averaging the distance
        # to do so, we need to convert the measures to strings
        # and then calculate the levenstein distance
        # between the two strings
        measure1 = measures_list[i]
        measure2 = measures_list[j]
        if len(measure1) < len(measure2):
            measure1, measure2 = measure2, measure1
        total_distance = 1
        for index, measure in enumerate(measure1):
            orig_measure = measure
            measure_to_compare = measure2[index] if index < len(measure2) else None
            old_stdout = sys.stdout  

            result = StringIO()
            sys.stdout = result
            orig_measure.measure.show('text') if orig_measure is not None else None
            orig_measure_string = result.getvalue()
            
            result = StringIO()
            sys.stdout = result
            measure_to_compare.measure.show('text') if measure_to_compare is not None else None
            measure_to_compare_string = result.getvalue()

            sys.stdout = old_stdout
            
            if orig_measure is None or measure_to_compare is None:
                continue
            if orig_measure is None:
                total_distance += len()
                continue
            if measure_to_compare is None:
                total_distance += len()
                continue
            
            total_distance += lev.distance(orig_measure_string, measure_to_compare_string)
        similarity_matrix[i][j] = total_distance

# the similarity matrix now has the distance between every measure
# transform the distance in each cell to a similarity
# by subtracting the distance from the maximum distance
# and then dividing by the maximum distance
similarity_matrix = 1 / similarity_matrix
similarity_matrix
        

Calculating similarity for measure 0

Calculating similarity for measure 1

Calculating similarity for measure 2

Calculating similarity for measure 3

Calculating similarity for measure 4



array([[1.        , 0.00129032, 0.00122399, 0.00107527, 0.00146199],
       [0.00129032, 1.        , 0.00332226, 0.0030581 , 0.0030303 ],
       [0.00122399, 0.00332226, 1.        , 0.00344828, 0.0025974 ],
       [0.00107527, 0.0030581 , 0.00344828, 1.        , 0.00207039],
       [0.00146199, 0.0030303 , 0.0025974 , 0.00207039, 1.        ]])

In [ ]:
# get the most similar measure to the first measure
# by getting the index of the maximum value in the first row
# and then getting the measure at that index
most_similar_measure_index = np.argmax(similarity_matrix[0])
most_similar_measure_index

0

In [ ]:
# convert the similarity matrix to a transition matrix
# where each cell in row i is the probability to transition from measure i to measure j
# and the probability to transition to the same measure is 0
# the most likely transition should be to the next measure
# so the probability to transition to the next measure should be the highest
# and the rest of the probabilities should be distributed according to the similarity to the next measure
# so the probability to transition from measure i to measure j should be the similarity between measure i + 1 and measure j

# the transition matrix should be a square matrix with the same size as the similarity matrix
transition_matrix = np.zeros((len(similarity_matrix), len(similarity_matrix)))
for i in range(len(similarity_matrix)):
    for j in range(len(similarity_matrix)):
        if i == j:
            continue
        transition_matrix[i][j] = similarity_matrix[j][i + 1]  if i + 1 < len(similarity_matrix) else similarity_matrix[j][0]
    transition_matrix[i] = transition_matrix[i] / np.sum(transition_matrix[i])



In [ ]:
transition_matrix

array([[0.        , 0.99067707, 0.00329129, 0.00302959, 0.00300205],
       [0.00121516, 0.        , 0.9927828 , 0.00342339, 0.00257866],
       [0.00106864, 0.00303925, 0.        , 0.99383448, 0.00205763],
       [0.0014517 , 0.00300897, 0.00257912, 0.        , 0.99296022],
       [0.99642326, 0.00128571, 0.00121961, 0.00107142, 0.        ]])

In [ ]:
# save the transition matrix to a file
np.savetxt('transition_matrix.csv', transition_matrix, delimiter=',')

In [ ]:
# load the transition matrix from a file
transition_matrix = np.loadtxt('transition_matrix.csv', delimiter=',')

In [ ]:
def new_add_measure(song_, previous_index):
    # get the number of measures in the song
    measure_num = len(song_.parts[0].getElementsByClass(m21.stream.Measure))
    # use the transition matrix to get the next measure
    next_measure_index = np.random.choice(len(transition_matrix[previous_index]), p=transition_matrix[previous_index])
    measures_in_order = measures_list[next_measure_index]
    for index, measure in enumerate(measures_in_order):
        if measure is None:
            continue
        if index == 0 and measure_num == 0:
            parts[index].insert(0, instrument_association[instrument])
        newMeasure = copy.deepcopy(measure.measure)
        parts[index].insert(measure_num * 4, newMeasure)
    return song_, next_measure_index

In [ ]:
song = m21.stream.Score()
parts = [m21.stream.Part() for _ in range(len(instrument_association))]
for index, instrument in enumerate(instruments):
    song.insert(0, parts[index])

current_new_measure = 0
for i in range(len(measures_list)):
    song, current_new_measure = new_add_measure(song, current_new_measure)
    print(f'Added measure {current_new_measure}')
song.write('midi', fp='test.mid')

Added measure 1
Added measure 2
Added measure 3
Added measure 4
Added measure 0


'test.mid'

In [ ]:
# write measures_list to a file with pickle
with open('measures_list.pkl', 'wb') as f:
    pickle.dump(measures_list, f)

In [ ]:
# read measures_list from a file with pickle
with open('measures_list.pkl', 'rb') as f:
    measures_list = pickle.load(f)

In [ ]:
len(measures_list)

352